In [1]:
# coding=utf-8
import pandas as pd
import csv  # Import csv
import tweepy
import re
import string
import nltk
import pandas as pd
from nltk import FreqDist
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import numpy as np
from sinling import SinhalaTokenizer, POSTagger
#pd.set_option('display.max_colwidth',1000)
tokenizer = SinhalaTokenizer()

In [2]:
#!/usr/bin/env python
# -*- coding:utf-8 -*-

import codecs
df = pd.read_csv('FinalListPreprocessedDataNew.csv', header = None, encoding='utf-8', engine='python')
df.columns = ['ID','User','CREATED_AT','LABEL','stemming_words','LIKE COUNT','FOLLOWERS COUNT','RETWEET COUNT']
df = df.dropna()
df.head()

,ID,User,CREATED_AT,LABEL,stemming_words,LIKE COUNT,FOLLOWERS COUNT,RETWEET COUNT
0,Unnamed: 0,User,CREATED_AT,LABEL,STEMMING WORDS,LIKE COUNT,FOLLOWERS COUNT,RETWEET COUNT
1,1,1.43633E+18,9/10/2021 13:53,Sport,ශ්රී ලංකා හා දකුණු අප්රිකාව අතර අද පැවැත්වී නි...,0,116,0
2,2,1.43632E+18,9/10/2021 13:09,Sport,අද තරඟ ක්රීඩා කරන කන්ඩායම දෙක චන්දි ඉන්,3,1344,0
3,3,1.43618E+18,9/10/2021 4:11,Sport,ක්රීඩාවෙන් ජාත්යන්තර ජයගත් ඔබේ කතාව අනාගත පරම්...,0,373,0
4,4,1.43614E+18,9/10/2021 1:30,Sport,ඒසේනම් ප්රිය ක්රීඩා ලෝලී හිතවතුනි අද තමා දවස,5,88,0


In [3]:
tokenizer = SinhalaTokenizer()
def clean_text(text):
    tokenized_sentences = [tokenizer.tokenize(f'{ss}.') for ss in tokenizer.split_sentences(text)]
    tagger = POSTagger()
    pos_tags =tagger.predict(tokenized_sentences) 
    return pos_tags 



In [4]:
df['cleaned_msg'] = df['stemming_words'].apply(lambda x: clean_text(x))
df.head()

,ID,User,CREATED_AT,LABEL,stemming_words,LIKE COUNT,FOLLOWERS COUNT,RETWEET COUNT,cleaned_msg
0,Unnamed: 0,User,CREATED_AT,LABEL,STEMMING WORDS,LIKE COUNT,FOLLOWERS COUNT,RETWEET COUNT,"[[(STEMMING, NNC), (WORDS, VFM), (., FS)]]"
1,1,1.43633E+18,9/10/2021 13:53,Sport,ශ්රී ලංකා හා දකුණු අප්රිකාව අතර අද පැවැත්වී නි...,0,116,0,"[[(ශ්රී, NNP), (ලංකා, NNP), (හා, CC), (දකුණු, ..."
2,2,1.43632E+18,9/10/2021 13:09,Sport,අද තරඟ ක්රීඩා කරන කන්ඩායම දෙක චන්දි ඉන්,3,1344,0,"[[(අද, NNC), (තරඟ, NCV), (ක්රීඩා, VNF), (කරන, ..."
3,3,1.43618E+18,9/10/2021 4:11,Sport,ක්රීඩාවෙන් ජාත්යන්තර ජයගත් ඔබේ කතාව අනාගත පරම්...,0,373,0,"[[(ක්රීඩාවෙන්, NNC), (ජාත්යන්තර, NNC), (ජයගත්,..."
4,4,1.43614E+18,9/10/2021 1:30,Sport,ඒසේනම් ප්රිය ක්රීඩා ලෝලී හිතවතුනි අද තමා දවස,5,88,0,"[[(ඒසේනම්, NNP), (ප්රිය, NNP), (ක්රීඩා, NNP), ..."


In [5]:
def pos_tag(data):
    result = []
    if type(data) == tuple:
        return data[1]

    if type(data) == list:
        for inner in data:
            result.append(pos_tag(inner))
    return result 

In [6]:
df['pos_tagging'] = df['cleaned_msg'].apply(lambda x: pos_tag(x))
df.head()

,ID,User,CREATED_AT,LABEL,stemming_words,LIKE COUNT,FOLLOWERS COUNT,RETWEET COUNT,cleaned_msg,pos_tagging
0,Unnamed: 0,User,CREATED_AT,LABEL,STEMMING WORDS,LIKE COUNT,FOLLOWERS COUNT,RETWEET COUNT,"[[(STEMMING, NNC), (WORDS, VFM), (., FS)]]","[[NNC, VFM, FS]]"
1,1,1.43633E+18,9/10/2021 13:53,Sport,ශ්රී ලංකා හා දකුණු අප්රිකාව අතර අද පැවැත්වී නි...,0,116,0,"[[(ශ්රී, NNP), (ලංකා, NNP), (හා, CC), (දකුණු, ...","[[NNP, NNP, CC, NNP, NNP, POST, NNC, VNF, RB, ..."
2,2,1.43632E+18,9/10/2021 13:09,Sport,අද තරඟ ක්රීඩා කරන කන්ඩායම දෙක චන්දි ඉන්,3,1344,0,"[[(අද, NNC), (තරඟ, NCV), (ක්රීඩා, VNF), (කරන, ...","[[NNC, NCV, VNF, VP, NNC, NUM, NNC, POST, FS]]"
3,3,1.43618E+18,9/10/2021 4:11,Sport,ක්රීඩාවෙන් ජාත්යන්තර ජයගත් ඔබේ කතාව අනාගත පරම්...,0,373,0,"[[(ක්රීඩාවෙන්, NNC), (ජාත්යන්තර, NNC), (ජයගත්,...","[[NNC, NNC, VP, PRP, NNC, JJ, NNC, NNC, VP, AU..."
4,4,1.43614E+18,9/10/2021 1:30,Sport,ඒසේනම් ප්රිය ක්රීඩා ලෝලී හිතවතුනි අද තමා දවස,5,88,0,"[[(ඒසේනම්, NNP), (ප්රිය, NNP), (ක්රීඩා, NNP), ...","[[NNP, NNP, NNP, NNP, NNP, NNC, PRP, NNC, FS]]"


In [7]:
import gensim
from gensim.models import Word2Vec
import nltk
from nltk.corpus import stopwords
import re

# My work

In [8]:
def get_pos_vec(x):
    w2v_model = Word2Vec(min_count=1,window=2,vector_size=5,sample=1e-5,alpha=0.01,min_alpha=0.0007,negative=0,workers=2)
    w2v_model.build_vocab(x, progress_per=1)
    w2v_model.train(x, total_examples=w2v_model.corpus_count, epochs=3, report_delay=1)
    
    for index, key in enumerate(w2v_model.wv.key_to_index):
        vec = w2v_model.wv[key]
    return vec

In [9]:
df['pos_vec'] = df['pos_tagging'].apply(lambda x: get_pos_vec(x))

In [10]:
def get_word_vec(text):
    x = [tokenizer.tokenize(f'{ss}.') for ss in tokenizer.split_sentences(text)]
    w2v_model = Word2Vec(min_count=1,window=2,vector_size=5,sample=1e-5,alpha=0.01,min_alpha=0.0007,negative=0,workers=2)
    w2v_model.build_vocab(x, progress_per=1)
    w2v_model.train(x, total_examples=w2v_model.corpus_count, epochs=3, report_delay=1)
    
    for index, key in enumerate(w2v_model.wv.key_to_index):
        vec = w2v_model.wv[key]
    return vec

In [11]:
df['word_vec'] = df['stemming_words'].apply(lambda x: get_word_vec(x))

In [12]:
#get sum of columns

dflist = ['pos_vec','word_vec']
df['sums'] = df[dflist].sum(axis=1)

In [13]:
df.head()

,ID,User,CREATED_AT,LABEL,stemming_words,LIKE COUNT,FOLLOWERS COUNT,RETWEET COUNT,cleaned_msg,pos_tagging,pos_vec,word_vec,sums
0,Unnamed: 0,User,CREATED_AT,LABEL,STEMMING WORDS,LIKE COUNT,FOLLOWERS COUNT,RETWEET COUNT,"[[(STEMMING, NNC), (WORDS, VFM), (., FS)]]","[[NNC, VFM, FS]]","[0.1476101, -0.03066945, -0.09073229, 0.131081...","[0.1476101, -0.03066945, -0.09073229, 0.131081...","[0.2952202, -0.0613389, -0.18146458, 0.2621620..."
1,1,1.43633E+18,9/10/2021 13:53,Sport,ශ්රී ලංකා හා දකුණු අප්රිකාව අතර අද පැවැත්වී නි...,0,116,0,"[[(ශ්රී, NNP), (ලංකා, NNP), (හා, CC), (දකුණු, ...","[[NNP, NNP, CC, NNP, NNP, POST, NNC, VNF, RB, ...","[0.10911588, 0.16691904, -0.029074812, -0.1841...","[0.02175293, -0.031524517, 0.04393344, -0.1576...","[0.1308688, 0.13539451, 0.014858628, -0.341794..."
2,2,1.43632E+18,9/10/2021 13:09,Sport,අද තරඟ ක්රීඩා කරන කන්ඩායම දෙක චන්දි ඉන්,3,1344,0,"[[(අද, NNC), (තරඟ, NCV), (ක්රීඩා, VNF), (කරන, ...","[[NNC, NCV, VNF, VP, NNC, NUM, NNC, POST, FS]]","[-0.15023164, -0.018600846, 0.19076237, -0.146...","[-0.192071, 0.10014586, -0.17519173, -0.087836...","[-0.34230265, 0.08154502, 0.015570641, -0.2342..."
3,3,1.43618E+18,9/10/2021 4:11,Sport,ක්රීඩාවෙන් ජාත්යන්තර ජයගත් ඔබේ කතාව අනාගත පරම්...,0,373,0,"[[(ක්රීඩාවෙන්, NNC), (ජාත්යන්තර, NNC), (ජයගත්,...","[[NNC, NNC, VP, PRP, NNC, JJ, NNC, NNC, VP, AU...","[-0.16315837, 0.089915946, -0.08274154, 0.0164...","[-0.17239375, 0.07331476, 0.103797674, 0.11483...","[-0.33555213, 0.16323072, 0.02105613, 0.131329..."
4,4,1.43614E+18,9/10/2021 1:30,Sport,ඒසේනම් ප්රිය ක්රීඩා ලෝලී හිතවතුනි අද තමා දවස,5,88,0,"[[(ඒසේනම්, NNP), (ප්රිය, NNP), (ක්රීඩා, NNP), ...","[[NNP, NNP, NNP, NNP, NNP, NNC, PRP, NNC, FS]]","[-0.03632035, 0.057531595, 0.019837474, -0.165...","[-0.192071, 0.10014586, -0.17519173, -0.087836...","[-0.22839135, 0.15767746, -0.15535426, -0.2535..."


# SVC

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn import metrics

X = df['sums'].to_numpy()
X = X.reshape(-1,1)
X.shape
X = np.concatenate(np.concatenate(X, axis = 0),axis = 0).reshape(-1,5)
y=df['LABEL']


X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape


clf = LinearSVC()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)


print(metrics.confusion_matrix(y_test,y_pred))

print(metrics.classification_report(y_test,y_pred))

[[  0   0  75   2   0]
 [  0   0  54   2   0]
 [  0   0 119   1   1]
 [  0   0  69   1   0]
 [  0   0  74   3   0]]
               precision    recall  f1-score   support

        Crime       0.00      0.00      0.00        77
Entertainment       0.00      0.00      0.00        56
        Other       0.30      0.98      0.46       121
     Politics       0.11      0.01      0.03        70
        Sport       0.00      0.00      0.00        77

     accuracy                           0.30       401
    macro avg       0.08      0.20      0.10       401
 weighted avg       0.11      0.30      0.14       401



c:\users\chandima samarakoon\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\chandima samarakoon\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\chandima samarakoon\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to cont

# Logestic Regression

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn import metrics



X = df['sums'].to_numpy()
X = X.reshape(-1,1)
X.shape
X = np.concatenate(np.concatenate(X, axis = 0),axis = 0).reshape(-1,5)
y=df['LABEL']


X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape


clf = LogisticRegression()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)


print(metrics.confusion_matrix(y_test,y_pred))

print(metrics.classification_report(y_test,y_pred))


[[  0   0  75   2   0]
 [  0   0  54   2   0]
 [  0   0 119   1   1]
 [  0   0  69   1   0]
 [  0   0  74   3   0]]
               precision    recall  f1-score   support

        Crime       0.00      0.00      0.00        77
Entertainment       0.00      0.00      0.00        56
        Other       0.30      0.98      0.46       121
     Politics       0.11      0.01      0.03        70
        Sport       0.00      0.00      0.00        77

     accuracy                           0.30       401
    macro avg       0.08      0.20      0.10       401
 weighted avg       0.11      0.30      0.14       401



c:\users\chandima samarakoon\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\chandima samarakoon\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\chandima samarakoon\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to cont

# KNN

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn import metrics

X = df['sums'].to_numpy()
X = X.reshape(-1,1)
X.shape
X = np.concatenate(np.concatenate(X, axis = 0),axis = 0).reshape(-1,5)
y=df['LABEL']


X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape


clf = KNeighborsClassifier()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)


print(metrics.confusion_matrix(y_test,y_pred))

print(metrics.classification_report(y_test,y_pred))

[[10 26 28  8  5]
 [10 17 21  5  3]
 [22 28 50 11 10]
 [10 25 18 13  4]
 [20 18 23 11  5]]
               precision    recall  f1-score   support

        Crime       0.14      0.13      0.13        77
Entertainment       0.15      0.30      0.20        56
        Other       0.36      0.41      0.38       121
     Politics       0.27      0.19      0.22        70
        Sport       0.19      0.06      0.10        77

     accuracy                           0.24       401
    macro avg       0.22      0.22      0.21       401
 weighted avg       0.24      0.24      0.23       401



# sgd 

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report
from sklearn import metrics

X = df['sums'].to_numpy()
X = X.reshape(-1,1)
X.shape
X = np.concatenate(np.concatenate(X, axis = 0),axis = 0).reshape(-1,5)
y=df['LABEL']


X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape


clf = SGDClassifier()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)


print(metrics.confusion_matrix(y_test,y_pred))

print(metrics.classification_report(y_test,y_pred))

[[ 0 63  1 13  0]
 [ 0 48  0  8  0]
 [ 0 93  0 28  0]
 [ 0 59  0 11  0]
 [ 0 66  1 10  0]]
               precision    recall  f1-score   support

        Crime       0.00      0.00      0.00        77
Entertainment       0.15      0.86      0.25        56
        Other       0.00      0.00      0.00       121
     Politics       0.16      0.16      0.16        70
        Sport       0.00      0.00      0.00        77

     accuracy                           0.15       401
    macro avg       0.06      0.20      0.08       401
 weighted avg       0.05      0.15      0.06       401



c:\users\chandima samarakoon\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\chandima samarakoon\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\chandima samarakoon\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to cont